<a href="https://colab.research.google.com/github/Mike-Xie/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Mike_Xie_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel



In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-11-04 20:28:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-11-04 20:28:50 (280 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [7]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

congress = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers,
                 na_values="?")

congress = congress.replace({'y':1, 'n':0})
congress.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [16]:
dem = congress[congress['party'] == 'democrat']
print(dem.shape)
dem.head()

(267, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [18]:
rep = congress[congress['party'] == 'republican']
print(rep.shape)
rep.head()

(168, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
def two_sample_t_test(column):
  tstat, pvalue = ttest_ind(rep[column], dem[column], nan_policy='omit')
  #if(pvalue < 0.05):
    #print("We reject the null hypothesis for" +column)
  # return(ttest_ind(rep[column], dem[column], nan_policy='omit'))
  return(tstat, pvalue)

In [67]:
two_sample_t_test('south-africa') #ok it works

(-6.849454815841208, 3.652674361672226e-11)

In [73]:
issues_slice = column_headers[1:]
issues_slice

['handicapped-infants',
 'water-project',
 'budget',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'immigration',
 'synfuels',
 'education',
 'right-to-sue',
 'crime',
 'duty-free']

In [79]:
# can do the same with a slice of column_headers (above)
# issues = ['handicapped-infants','water-project',
#                           'budget','physician-fee-freeze', 'el-salvador-aid',
#                           'religious-groups','anti-satellite-ban',
#                           'aid-to-contras','mx-missile','immigration',
#                           'synfuels', 'education', 'right-to-sue','crime','duty-free',
#                           'south-africa']
# let's turn this into a function, I think it's more readable in normal form if it's going to be long
# [two_sample_t_test(issue) for issue in issues]

t_vals = {}
p_vals = {}

for column in issues_slice:
  tstat, pvalue = ttest_ind(rep[column], dem[column], nan_policy='omit')
  print("The two sample T-Test result in "+ column + " is:")
  print(two_sample_t_test(column))
  t_vals[column] = tstat
  p_vals[column] = pvalue
  # make these dictionaries instead
print("\n")
print(t_vals)
print(p_vals)


The two sample T-Test result in handicapped-infants is:
(-9.205264294809222, 1.613440327937243e-18)
The two sample T-Test result in water-project is:
(0.08896538137868286, 0.9291556823993485)
The two sample T-Test result in budget is:
(-23.21277691701378, 2.0703402795404463e-77)
The two sample T-Test result in physician-fee-freeze is:
(49.36708157301406, 1.994262314074344e-177)
The two sample T-Test result in el-salvador-aid is:
(21.13669261173219, 5.600520111729011e-68)
The two sample T-Test result in religious-groups is:
(9.737575825219457, 2.3936722520597287e-20)
The two sample T-Test result in anti-satellite-ban is:
(-12.526187929077842, 8.521033017443867e-31)
The two sample T-Test result in aid-to-contras is:
(-18.052093200819733, 2.82471841372357e-54)
The two sample T-Test result in mx-missile is:
(-16.437503268542994, 5.03079265310811e-47)
The two sample T-Test result in immigration is:
(1.7359117329695164, 0.08330248490425066)
The two sample T-Test result in synfuels is:
(-8.29

**Load and clean the data (or determine the best method to drop observations when running tests)
Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference **

1) Null Hypothesis (Boring): There is no difference between average voting rates for each bill between Democrats & Republicans.
List of Bills:
['handicapped-infants',
 'water-project',
 'budget',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'immigration',
 'synfuels',
 'education',
 'right-to-sue',
 'crime',
 'duty-free']

$\bar{x}_{1} ==\bar{x}_{2}$ for each bill in the data set.

Where $\bar{x}_{1}$ is the mean of Republican Votes and $\bar{x}_{2}$ is the mean of Democrat Votes.

2) Alternative Hypothesis:
$\bar{x}_{1} \neq\bar{x}_{2}$ 

3) Confidence Level: The probability of seeing a true result in spite of random variability. Typically 95% or 2 standard deviations.

4) T-Statistics: 
[-9.205264294809222, 0.08896538137868286, -23.21277691701378, 49.36708157301406, 21.13669261173219, 9.737575825219457, -12.526187929077842, -18.052093200819733, -16.437503268542994, 1.7359117329695164, -8.293603989407588, 20.500685724563073, 13.51064251060933, 16.342085656197696, -12.853146132542978]

5) P-Values:
[1.613440327937243e-18, 0.9291556823993485, 2.0703402795404463e-77, 1.994262314074344e-177, 5.600520111729011e-68, 2.3936722520597287e-20, 8.521033017443867e-31, 2.82471841372357e-54, 5.03079265310811e-47, 0.08330248490425066, 1.5759322301054064e-15, 1.8834203990450192e-64, 1.2278581709672758e-34, 9.952342705606092e-47, 5.997697174347365e-32]

6) Conclusions:

We reject the null hypothesis if p-value is < 0.01 and so we fail to reject the null hypothesis for Immigration (.083) and the water project (.929) and we reject it for everything else.

6a) The only issue with p-value greater than 0.1 is the water project.



In [77]:
print('$\bar{x}_{1} ==\bar{x}_{2}$') 

$ar{x}_{1} ==ar{x}_{2}$


In [66]:
rep['budget'].mean()

0.13414634146341464

In [65]:
dem['budget'].mean()

0.8884615384615384

In [0]:
# can't figure out how to print with the special formating for symbols

# print('''

# 1) Null Hypothesis (Boring): There is no difference between average voting rates for each bill between Democrats & Republicans.

# $\bar{x}_{1} ==\bar{x}_{2}$ for each bill in the data set.

# Where $\bar{x}_{1}$ is the mean of Republican Votes and $\bar{x}_{2}$ is the mean of Democrat Votes.

# 2) Alternative Hypothesis:
# $\bar{x}_{1} \neq\bar{x}_{2}$ 

# 3) Confidence Level: The probability of seeing a true result in spite of random variability. Typically 95% or 2 standard deviations.

# 4) T-Statistics:

# ''')